# EFM Large-Scale Structure (LSS) & Clustering Validation (Dimensionless, HPC Optimized)

This notebook performs a high-resolution simulation of Large-Scale Structure (LSS) formation within the Eholoko Fluxon Model (EFM) framework. Crucially, this simulation operates entirely in **dimensionless units**, consistent with the core theoretical foundation of EFM as implied by the parameter choices in its foundational papers (e.g., `c=1.0`, `G=1.0`). Physical interpretations (e.g., Mpc) will be derived from the emergent dimensionless scales during post-processing.

This revised approach addresses the numerical instabilities encountered in previous attempts due to mixing physical and dimensionless units in the core simulation loop. The simulation is optimized for multi-GPU execution using PyTorch's DistributedDataParallel (DDP) for efficiency.

## EFM Theoretical Grounding for LSS:

1.  **Single Scalar Field (φ):** All phenomena, including cosmic structure, emerge from the dynamics of this fundamental field [1, 2].
2.  **NLKG Equation with EFM Self-Gravity:** The evolution of φ is governed by a specific Nonlinear Klein-Gordon equation. For LSS, the dominant terms are:
    *   `c²∇²φ`: Spatial interaction/propagation.
    *   `m²φ`: The mass term for the background field. **For LSS, the paper 'Unifying Cosmic Structure' (Ref [4]) specifies `m=1.0` for its core equation.** Mass is an *emergent* property of localized ehokolons (solitons), not solely an intrinsic property of the background field.
    *   `gφ³`, `ηφ⁵`: Nonlinear self-interaction terms crucial for preventing linear dispersion and driving the formation and stabilization of cosmic structures [4].
    *   `8πGkφ²`: The EFM self-gravity term, where the field's own density (`kφ²`) acts as the source for emergent gravitational interactions, replacing spacetime curvature [5].
    *   `αφ(∂φ/∂t)⋅∇φ`, `δ(∂φ/∂t)²φ`: State-dependent dynamical friction and dissipation terms, important for system stability and evolution [6]. These terms are implemented based on common EFM usage for scalar fields.
3.  **Harmonic Density States (HDS):** While not explicitly simulated as transitions here, the HDS framework dictates the emergence of characteristic clustering scales (~147 Mpc and ~628 Mpc) that this simulation aims to reproduce [7].
4.  **Initial Conditions:** Small amplitude Gaussian noise represents primordial fluctuations from which large-scale structures emerge [4].

## Objectives:

-   Simulate 3D LSS formation on a 300³ grid for 50,000 timesteps using dimensionless EFM parameters.
-   Confirm numerical stability of the dimensionless approach.
-   Compute power spectrum P(k) and correlation function ξ(r) from the dimensionless results.
-   Infer physical clustering scales (147 Mpc, 628 Mpc) by analyzing the emergent dimensionless scales.

## Google Drive Setup (for Colab)

To ensure data and plots are saved to and retrieved from your Google Drive, please execute the following cell to mount your Drive. If you are running this notebook in a non-Colab environment, adjust the `data_path_lss` variable accordingly.


In [ ]:
# This cell is specific to Google Colab environments
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except ImportError:
    print("Not in Google Colab environment. Skipping Google Drive mount.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}. Please ensure you're logged in and have granted permissions.")


In [ ]:
import os
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.multiprocessing as mp
import gc
import psutil
from tqdm.notebook import tqdm # Use tqdm.notebook for Jupyter environments
import numpy as np
import time
from datetime import datetime
from scipy.fft import fftn, fftfreq, ifftn # Using scipy for CPU-based FFT for final analysis
import torch.nn.functional as F
import torch.cuda.amp as amp # For Automatic Mixed Precision
import matplotlib.pyplot as plt # For plotting
import glob

# Environment setup for PyTorch CUDA memory management
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print(f"PyTorch version: {torch.__version__}")
num_gpus_available = torch.cuda.device_count()
available_devices_list = [torch.device(f'cuda:{i}') for i in range(num_gpus_available)]
print(f"Number of GPUs available: {num_gpus_available}, Target devices for DDP: {available_devices_list}")
if num_gpus_available > 0:
    current_gpu_device = torch.device('cuda:0')
    print(f"Using GPU 0: {torch.cuda.get_device_name(current_gpu_device)}, VRAM: {torch.cuda.get_device_properties(current_gpu_device).total_memory / 1e9:.2f} GB")
else:
    current_gpu_device = torch.device('cpu')
    print("No GPU available, running on CPU. Performance may be limited.")
print(f"System RAM: {psutil.virtual_memory().total / 1e9:.2f} GB")

# Define paths for checkpoints and data/plots - Adjusted for Google Drive
checkpoint_path_lss = '/content/drive/My Drive/EFM_Simulations/checkpoints/LSS_DIMLESS_HPC_Opt_v1/'
data_path_lss = '/content/drive/My Drive/EFM_Simulations/data/LSS_DIMLESS_HPC_Opt_v1/'
os.makedirs(checkpoint_path_lss, exist_ok=True)
os.makedirs(data_path_lss, exist_ok=True)
print(f"LSS Checkpoints will be saved to: {checkpoint_path_lss}")
print(f"LSS Data/Plots will be saved to: {data_path_lss}")


## Configuration for LSS Simulation (Dimensionless Units, HPC Optimized)

Parameters are set according to EFM principles for LSS as dimensionless units. `N` and `T_steps` are increased for a higher-resolution, longer run.

### Parameter Derivation and EFM Justification (Dimensionless):

*   **`N`**: Grid size (N x N x N). **Increased to `300` for this higher-resolution A100 run.**
*   **`L_sim_unit`**: Physical size of the simulation box in **dimensionless simulation units**. `10.0` is a common choice for such dimensionless systems. This `L` will later be scaled to a physical size (e.g., Mpc) based on emergent structures.
*   **`dx_sim_unit`**: Spatial step in dimensionless units. Calculated as `L_sim_unit / N`.
*   **`c_sim_unit`**: Speed of light in **dimensionless simulation units**. Set to `1.0` as per EFM papers' implied dimensionless constants (e.g., 'Fluxonic Cosmology' Section 2.1) for internal consistency.
*   **`dt_cfl_factor`**: Courant-Friedrichs-Lewy (CFL) condition factor. `0.001` is a robust value for stability in dimensionless `c=1` systems with nonlinear dynamics.
*   **`T_steps`**: Total simulation steps. **Increased to `50000` for a longer evolution to allow structure formation.**
*   **`m_sim_unit_inv` (m in m²φ)**: Mass term coefficient in dimensionless units. **Set to `1.0` as per 'Unifying Cosmic Structure' paper Section 2 [4].**
*   **`g_sim` (g in gφ³)**: Cubic nonlinearity coefficient in dimensionless units. **Set to `0.1` as per 'Unifying Cosmic Structure' paper Section 2 [4].**
*   **`eta_sim` (η in ηφ⁵)**: Quintic nonlinearity coefficient in dimensionless units. **Set to `0.01` as per 'Unifying Cosmic Structure' paper Section 2 [4].**
*   **`k_efm_gravity_coupling` (k in 8πGkφ²)**: Coupling constant for the EFM self-gravity term in dimensionless units. **Set to `0.005` as per 'Unifying Cosmic Structure' paper Section 2 [4].**
*   **`G_sim_unit` (G in 8πGkφ²)**: Gravitational constant in dimensionless simulation units. **Set to `1.0` [4].**
*   **`alpha_sim` (α in αφ(∂φ/∂t)⋅∇φ)**: State parameter in dimensionless units. **Set to `0.7` for S/T state dynamics as per 'Unifying Cosmic Structure' paper Section 2 [4].**
*   **`delta_sim` (δ in δ(∂φ/∂t)²φ)**: Dissipation term in dimensionless units. **Set to `0.0002` as per 'Unifying Cosmic Structure' paper Section 2 [4].**
*   **`initial_noise_amplitude`**: Amplitude of initial Gaussian noise in dimensionless units. `1.0e-6` is a common choice for seeding initial perturbations.


In [ ]:
config_lss_run = {}
config_lss_run['N'] = 300  # Grid size (N x N x N) - Increased to 300
config_lss_run['L_sim_unit'] = 10.0  # Dimensionless box size
config_lss_run['dx_sim_unit'] = config_lss_run['L_sim_unit'] / config_lss_run['N'] # Dimensionless spatial step

config_lss_run['c_sim_unit'] = 1.0  # Dimensionless speed of light (as per EFM papers)
config_lss_run['dt_cfl_factor'] = 0.001 # Robust CFL factor for dimensionless c=1 system
config_lss_run['dt_sim_unit'] = config_lss_run['dt_cfl_factor'] * config_lss_run['dx_sim_unit'] / config_lss_run['c_sim_unit']

config_lss_run['T_steps'] = 50000 # Total number of time steps - Increased to 50000

# EFM Parameters for LSS from 'Unifying Cosmic Structure' paper, Section 2
config_lss_run['m_sim_unit_inv'] = 1.0 # m=1.0 for LSS in this paper
config_lss_run['g_sim'] = 0.1          # g from paper
config_lss_run['eta_sim'] = 0.01         # eta from paper
config_lss_run['k_efm_gravity_coupling'] = 0.005 # k from paper
config_lss_run['G_sim_unit'] = 1.0 # G from paper
config_lss_run['alpha_sim'] = 0.7  # alpha for S/T state from paper
config_lss_run['delta_sim'] = 0.0002 # delta from paper

config_lss_run['initial_noise_amplitude'] = 1.0e-6 # Common initial noise amplitude

config_lss_run['run_id'] = (
    f"LSS_N{config_lss_run['N']}_T{config_lss_run['T_steps']}_" +
    f"m{config_lss_run['m_sim_unit_inv']:.1e}_g{config_lss_run['g_sim']:.1e}_eta{config_lss_run['eta_sim']:.1e}_" +
    f"k{config_lss_run['k_efm_gravity_coupling']:.1e}_G{config_lss_run['G_sim_unit']:.1e}_alpha{config_lss_run['alpha_sim']:.1e}_delta{config_lss_run['delta_sim']:.1e}_" +
    f"CFL{config_lss_run['dt_cfl_factor']:.1e}_A100_DIMLESS_LSS_v2"
)

config_lss_run['history_every_n_steps'] = 500 # Frequency of calculating/storing diagnostics
config_lss_run['checkpoint_every_n_steps'] = 2000 # Frequency of saving intermediate checkpoints

print(f"--- EFM LSS Simulation Configuration ({config_lss_run['run_id']}) ---")
for key, value in config_lss_run.items():
    if isinstance(value, (float, np.float32, np.float64)):
        print(f"{key}: {value:.4g}")
    else:
        print(f"{key}: {value}")

print("\n--- Physical Scaling (for interpretation of dimensionless results) ---")
print("The simulation runs in dimensionless units. Physical scales (Mpc, Gyr) will be derived post-simulation.")
print(f"Dimensionless L: {config_lss_run['L_sim_unit']} units, dx: {config_lss_run['dx_sim_unit']:.4g} units")
print(f"Dimensionless dt: {config_lss_run['dt_sim_unit']:.4g} units")


## Mathematical Framework: EFM Nonlinear Klein-Gordon Equation for LSS

The core dynamics are governed by the specific NLKG equation from Section 2 of the 'Ehokolo Fluxon Model: Unifying Cosmic Structure, Non-Gaussianity, and Gravitational Waves Across Scales' paper [4]:

```
∂²φ/∂t² − c²∇²φ + m²φ + gφ³ + ηφ⁵ + δφ⁷ = 8πGkφ² + β(B × ∇φ) + αφ(∂φ/∂t)⋅∇φ
```
This is a general form. For LSS, simplified assumptions are often made as detailed in the paper's section 2. We will use the following terms, consistent with the cosmological model implemented across EFM papers:

```
φ_ddot = c²∇²φ - (m²φ + gφ³ + ηφ⁵) + αφ(∂φ/∂t)⋅∇φ + δ(∂φ/∂t)²φ + 8πGkφ²
```
The terms are:

*   `∂²φ/∂t²`: Second time derivative (acceleration).
*   `− c²∇²φ`: Spatial curvature/propagation term. This term drives wave propagation and is analogous to kinetic energy density from spatial gradients.
*   `+ m²φ + gφ³ + ηφ⁵`: Self-interaction potential terms, `V'(φ)`. These terms are derived from the potential `V(φ) = m²φ²/2 + gφ⁴/4 + ηφ⁶/6` and are crucial for the stability and formation of localized structures (ehokolons). For LSS, `m=1.0` in this specific paper [4].
*   `− αφ(∂φ/∂t)⋅∇φ`: State-dependent `α` term. On the RHS, `+ αφ(∂φ/∂t)⋅∇φ`. Interpreted as `α * φ * (∂φ/∂t) * |∇φ|^2` for scalar consistency, as this form is commonly applied for such dynamic terms in EFM scalar equations.
*   `− δ(∂φ/∂t)²φ`: Dissipation term. On the RHS, `+ δ(∂φ/∂t)²φ`. This form is used consistently for dissipation in other EFM papers [6]. The `δφ⁷` term from the general equation in [4] is not used here for consistency with the broader EFM cosmological framework.
*   `8πGkφ²`: The EFM self-gravity source term. This term drives the clustering of matter in LSS.

This equation drives the evolution of the φ field to form cosmic structures.

In [ ]:
class EFMLSSModule(nn.Module):
    """
    EFM Module for the NLKG equation for LSS, using dimensionless parameters.
    """
    def __init__(self, dx, m_sq, g, eta, k_gravity, G_gravity, c_sq, alpha_param, delta_param):
        super(EFMLSSModule, self).__init__()
        self.dx = dx
        self.m_sq = m_sq # Will be 0 for LSS
        self.g = g
        self.eta = eta
        self.k_gravity = k_gravity
        self.G_gravity = G_gravity
        self.c_sq = c_sq
        self.alpha_param = alpha_param
        self.delta_param = delta_param

        # 3D Laplacian stencil (7-point, order 2) for periodic boundary conditions
        stencil_np = np.array([[[0,0,0],[0,1,0],[0,0,0]],
                               [[0,1,0],[1,-6,1],[0,1,0]],
                               [[0,0,0],[0,1,0],[0,0,0]]], dtype=np.float32)
        self.stencil = torch.from_numpy(stencil_np / (dx**2))
        self.stencil = self.stencil.view(1, 1, 3, 3, 3)

    def conv_laplacian(self, phi_field):
        stencil_dev = self.stencil.to(phi_field.device)
        phi_reshaped = phi_field.unsqueeze(0).unsqueeze(0)
        phi_padded = F.pad(phi_reshaped, (1,1,1,1,1,1), mode='circular')
        laplacian = F.conv3d(phi_padded, stencil_dev, padding=0)
        return laplacian.squeeze(0).squeeze(0)

    def nlkg_derivative_lss(self, phi, phi_dot):
        """
        Computes time derivatives (phi_dot, phi_ddot) based on the EFM LSS NLKG equation.
        Equation: ∂²φ/∂t² − c²∇²φ + m²φ + gφ³ + ηφ⁵ − αφ(∂φ/∂t)⋅∇φ − δ(∂φ/∂t)²φ = 8πGkφ²
        Rearranging to solve for φ_ddot:
        φ_ddot = c²∇²φ - (m²φ + gφ³ + ηφ⁵) + αφ(∂φ/∂t)⋅∇φ + δ(∂φ/∂t)²φ + 8πGkφ²
        """
        lap_phi = self.conv_laplacian(phi)

        # V'(φ) = m²φ + gφ³ + ηφ⁵
        potential_force = self.m_sq * phi + \
                          self.g * torch.pow(phi, 3) + \
                          self.eta * torch.pow(phi, 5)

        # Term: αφ(∂φ/∂t)⋅∇φ (re-interpreting dot product as a scalar term |∇φ|^2 for NLKG scalar equation)
        grad_phi_x = (torch.roll(phi, shifts=-1, dims=0) - torch.roll(phi, shifts=1, dims=0)) / (2 * self.dx)
        grad_phi_y = (torch.roll(phi, shifts=-1, dims=1) - torch.roll(phi, shifts=1, dims=1)) / (2 * self.dx)
        grad_phi_z = (torch.roll(phi, shifts=-1, dims=2) - torch.roll(phi, shifts=1, dims=2)) / (2 * self.dx)
        grad_phi_abs_sq = grad_phi_x**2 + grad_phi_y**2 + grad_phi_z**2
        alpha_term = self.alpha_param * phi * phi_dot * grad_phi_abs_sq

        # Term: δ(∂φ/∂t)²φ
        delta_term = self.delta_param * torch.pow(phi_dot, 2) * phi

        # Source term: 8πGkφ²
        source_gravity = 8.0 * float(np.pi) * self.G_gravity * self.k_gravity * torch.pow(phi, 2)

        # Equation of motion: φ_ddot = c²∇²φ - V'(φ) + alpha_term + delta_term + source_gravity
        phi_ddot = self.c_sq * lap_phi - potential_force + alpha_term + delta_term + source_gravity
        
        return phi_dot, phi_ddot # Returns (dφ/dt, d²φ/dt²)

def update_phi_rk4_lss(phi_current: torch.Tensor, phi_dot_current: torch.Tensor,
                       dt: float, model_instance: EFMLSSModule) -> tuple[torch.Tensor, torch.Tensor]:
    """
    Updates phi and phi_dot using the RK4 method for one time step.
    """
    # Using Automatic Mixed Precision for performance on A100.
    with amp.autocast(dtype=torch.float16):
        k1_v, k1_a = model_instance.nlkg_derivative_lss(phi_current, phi_dot_current)
        
        phi_temp_k2 = phi_current + 0.5 * dt * k1_v
        phi_dot_temp_k2 = phi_dot_current + 0.5 * dt * k1_a
        k2_v, k2_a = model_instance.nlkg_derivative_lss(phi_temp_k2, phi_dot_temp_k2)
        
        phi_temp_k3 = phi_current + 0.5 * dt * k2_v
        phi_dot_temp_k3 = phi_dot_current + 0.5 * dt * k2_a
        k3_v, k3_a = model_instance.nlkg_derivative_lss(phi_temp_k3, phi_dot_temp_k3)
        
        phi_temp_k4 = phi_current + dt * k3_v
        phi_dot_temp_k4 = phi_dot_current + dt * k3_a
        k4_v, k4_a = model_instance.nlkg_derivative_lss(phi_temp_k4, phi_dot_temp_k4)
            
        phi_next = phi_current + (dt / 6.0) * (k1_v + 2*k2_v + 2*k3_v + k4_v)
        phi_dot_next = phi_dot_current + (dt / 6.0) * (k1_a + 2*k2_a + 2*k3_a + k4_a)

    # Explicitly clean up intermediate tensors
    del k1_v, k1_a, k2_v, k2_a, k3_v, k3_a, k4_v, k4_a
    del phi_temp_k2, phi_dot_temp_k2, phi_temp_k3, phi_dot_temp_k3, phi_temp_k4, phi_dot_temp_k4
    
    torch.cuda.synchronize(phi_current.device)
    gc.collect() 
    torch.cuda.empty_cache()
        
    return phi_next, phi_dot_next

def compute_total_energy_lss(phi: torch.Tensor, phi_dot: torch.Tensor,
                              m_sq_param: float, g_param: float, eta_param: float,
                              dx: float, c_sq_param: float, model_instance: EFMLSSModule) -> float:
    """
    Computes the total field energy based on the EFM Lagrangian for LSS (dimensionless units).
    Energy E = ∫ [1/2 (∂φ/∂t)² + 1/2 c²|∇φ|² + (m²φ²/2 + gφ⁴/4 + ηφ⁶/6)] dV
    """
    vol_element = dx**3
    total_energy = torch.tensor(0.0, device=phi.device, dtype=torch.float64)

    phi_f32 = phi.to(dtype=torch.float32)
    phi_dot_f32 = phi_dot.to(dtype=torch.float32)

    with amp.autocast(dtype=torch.float16):
        kinetic_density = 0.5 * torch.pow(phi_dot_f32, 2)
        potential_density = 0.5 * m_sq_param * torch.pow(phi_f32, 2) + \
                            0.25 * g_param * torch.pow(phi_f32, 4) + \
                            (1.0/6.0) * eta_param * torch.pow(phi_f32, 6)
        
        grad_phi_x = (torch.roll(phi_f32, shifts=-1, dims=0) - torch.roll(phi_f32, shifts=1, dims=0)) / (2 * dx)
        grad_phi_y = (torch.roll(phi_f32, shifts=-1, dims=1) - torch.roll(phi_f32, shifts=1, dims=1)) / (2 * dx)
        grad_phi_z = (torch.roll(phi_f32, shifts=-1, dims=2) - torch.roll(phi_f32, shifts=1, dims=2)) / (2 * dx)
        gradient_energy_density = 0.5 * c_sq_param * (grad_phi_x**2 + grad_phi_y**2 + grad_phi_z**2)

        total_energy_current_chunk = torch.sum(kinetic_density + potential_density + gradient_energy_density) * vol_element

    if torch.isnan(total_energy_current_chunk) or torch.isinf(total_energy_current_chunk):
        return float('nan')

    total_energy_val = total_energy_current_chunk.item()

    del phi_f32, phi_dot_f32, kinetic_density, potential_density, gradient_energy_density
    del grad_phi_x, grad_phi_y, grad_phi_z
    gc.collect()
    torch.cuda.empty_cache()

    return total_energy_val

def compute_power_spectrum_lss(phi_cpu_np_array: np.ndarray, k_val_range: list,
                               dx_val_param: float, N_grid_param: int) -> tuple[np.ndarray, np.ndarray]:
    """Computes the 3D power spectrum P(k) in dimensionless units."""
    if not isinstance(phi_cpu_np_array, np.ndarray):
        phi_cpu_np_array = phi_cpu_np_array.cpu().numpy() # Ensure it's a NumPy array on CPU

    phi_fft_transform = fftn(phi_cpu_np_array.astype(np.float32))
    # Power spectrum |F(φ)|² normalized by N_total_points^2 = (N^3)^2 = N^6 for density
    power_spectrum_raw_data = np.abs(phi_fft_transform)**2 / (N_grid_param**6)
    del phi_fft_transform
    gc.collect()

    # Create k-space grid
    kx_coords = fftfreq(N_grid_param, d=dx_val_param) * 2 * np.pi
    ky_coords = fftfreq(N_grid_param, d=dx_val_param) * 2 * np.pi
    kz_coords = fftfreq(N_grid_param, d=dx_val_param) * 2 * np.pi
    kxx_mesh, kyy_mesh, kzz_mesh = np.meshgrid(kx_coords, ky_coords, kz_coords, indexing='ij', sparse=True)
    k_magnitude_values = np.sqrt(kxx_mesh**2 + kyy_mesh**2 + kzz_mesh**2)
    del kxx_mesh, kyy_mesh, kzz_mesh, kx_coords, ky_coords, kz_coords
    gc.collect()

    # Binning the power spectrum
    # k_val_range here is in dimensionless units
    k_bins_def = np.linspace(k_val_range[0], k_val_range[1], 50) # 50 bins
    power_binned_values, _ = np.histogram(
        k_magnitude_values.ravel(), bins=k_bins_def,
        weights=power_spectrum_raw_data.ravel(), density=False
    )
    counts_in_bins, _ = np.histogram(k_magnitude_values.ravel(), bins=k_bins_def)
    
    power_binned_final = np.divide(power_binned_values, counts_in_bins, out=np.zeros_like(power_binned_values), where=counts_in_bins!=0)
    k_bin_centers_final = (k_bins_def[:-1] + k_bins_def[1:]) / 2
    
    del k_magnitude_values, power_spectrum_raw_data, counts_in_bins
    gc.collect()
    return k_bin_centers_final, power_binned_final

def compute_correlation_function_lss(phi_cpu_np_array: np.ndarray, dx_val_param: float,
                                     N_grid_param: int, L_box_param: float) -> tuple[np.ndarray, np.ndarray]:
    """Computes the 3D correlation function ξ(r) in dimensionless units."""
    if not isinstance(phi_cpu_np_array, np.ndarray):
        phi_cpu_np_array = phi_cpu_np_array.cpu().numpy()

    phi_fft_transform = fftn(phi_cpu_np_array.astype(np.float32))
    power_spectrum_raw_data = np.abs(phi_fft_transform)**2 # P(k) = |F(φ)|²
    del phi_fft_transform
    gc.collect()
    
    correlation_func_raw_data = ifftn(power_spectrum_raw_data).real / (N_grid_param**3) # Normalization for ξ(r)
    del power_spectrum_raw_data
    gc.collect()

    # Create r-space grid for binning (shifted for ξ(0) at center)
    indices_shifted = np.fft.ifftshift(np.arange(N_grid_param)) - (N_grid_param // 2) # Centered indices
    rx_coords = indices_shifted * dx_val_param
    ry_coords = indices_shifted * dx_val_param
    rz_coords = indices_shifted * dx_val_param
    rxx_mesh, ryy_mesh, rzz_mesh = np.meshgrid(rx_coords, ry_coords, rz_coords, indexing='ij', sparse=True)
    r_magnitude_values = np.sqrt(rxx_mesh**2 + ryy_mesh**2 + rzz_mesh**2)
    del rx_coords, ry_coords, rz_coords, rxx_mesh, ryy_mesh, rzz_mesh
    gc.collect()

    r_bins_def = np.linspace(0, L_box_param / 2, 50) # Bins up to half the box size
    corr_binned_values, _ = np.histogram(
        r_magnitude_values.ravel(), bins=r_bins_def,
        weights=correlation_func_raw_data.ravel()
    )
    counts_in_bins, _ = np.histogram(r_magnitude_values.ravel(), bins=r_bins_def)
    corr_binned_final = np.divide(corr_binned_values, counts_in_bins, out=np.zeros_like(corr_binned_values), where=counts_in_bins!=0)
    r_bin_centers_final = (r_bins_def[:-1] + r_bins_def[1:]) / 2
    
    del r_magnitude_values, correlation_func_raw_data, counts_in_bins
    gc.collect()
    return r_bin_centers_final, corr_binned_final


## Simulation Orchestration for EFM LSS (A100)

This section sets up the simulation loop, handles initial conditions, and records diagnostics. The simulation runs as a single process on the A100 GPU for simplicity, leveraging its computational power.


In [ ]:
def run_lss_simulation(config: dict, device: torch.device):
    """Main simulation loop for EFM LSS."""
    print(f"Initializing fields for EFM LSS simulation ({config['run_id']}) on {device}...")
    
    # Set seed for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)

    # Initial conditions: small amplitude Gaussian noise
    phi = torch.randn(config['N'], config['N'], config['N'], dtype=torch.float16, device=device) * config['initial_noise_amplitude']
    phi_dot = torch.zeros_like(phi, dtype=torch.float16, device=device)

    # Instantiate the EFM LSS Module
    efm_model = EFMLSSModule(
        dx=config['dx_sim_unit'], # Use dimensionless dx
        m_sq=config['m_sim_unit_inv']**2,
        g=config['g_sim'],
        eta=config['eta_sim'],
        k_gravity=config['k_efm_gravity_coupling'],
        G_gravity=config['G_sim_unit'], # Use dimensionless G
        c_sq=config['c_sim_unit']**2, # Use dimensionless c
        alpha_param=config['alpha_sim'], # Use LSS alpha
        delta_param=config['delta_sim'] # Use LSS delta
    ).to(device)
    efm_model.eval() # No training, so eval mode is appropriate

    # History tracking
    num_hist_points = config['T_steps'] // config['history_every_n_steps'] + 1
    energy_history = np.zeros(num_hist_points, dtype=np.float64)
    density_norm_history = np.zeros(num_hist_points, dtype=np.float64)
    hist_idx = 0

    # Record initial state diagnostics
    energy_history[hist_idx] = compute_total_energy_lss(phi, phi_dot, efm_model.m_sq, efm_model.g, efm_model.eta, efm_model.dx, efm_model.c_sq, efm_model)
    density_norm_history[hist_idx] = torch.sum(phi.to(torch.float32)**2).item() * config['k_efm_gravity_coupling']
    print(f"Initial State: Energy={energy_history[hist_idx]:.4g}, Density Norm={density_norm_history[hist_idx]:.4g}")
    hist_idx += 1

    sim_start_time = time.time()
    numerical_error = False

    for t_step in tqdm(range(config['T_steps']), desc=f"LSS Sim ({config['run_id']})"):
        # Check for numerical instability before update
        if torch.any(torch.isinf(phi)) or torch.any(torch.isnan(phi)) or \
           torch.any(torch.isinf(phi_dot)) or torch.any(torch.isnan(phi_dot)):
            print(f"\nERROR: NaN/Inf detected in fields BEFORE step {t_step + 1}! Stopping.")
            numerical_error = True
            break

        # RK4 update
        phi, phi_dot = update_phi_rk4_lss(phi, phi_dot, config['dt_sim_unit'], efm_model)

        # Check for numerical instability after update
        if torch.any(torch.isinf(phi)) or torch.any(torch.isnan(phi)):
            print(f"\nERROR: NaN/Inf detected in phi AFTER step {t_step + 1}! Stopping.")
            numerical_error = True
            break

        # Record diagnostics periodically
        if (t_step + 1) % config['history_every_n_steps'] == 0:
            if hist_idx < num_hist_points:
                current_energy = compute_total_energy_lss(phi, phi_dot, efm_model.m_sq, efm_model.g, efm_model.eta, efm_model.dx, efm_model.c_sq, efm_model)
                current_density_norm = torch.sum(phi.to(torch.float32)**2).item() * config['k_efm_gravity_coupling']

                energy_history[hist_idx] = current_energy
                density_norm_history[hist_idx] = current_density_norm
                
                tqdm.write(f"Step {t_step+1}: E={current_energy:.3e}, DN={current_density_norm:.3e}")
                if np.isnan(current_energy) or np.isinf(current_energy):
                    print(f"Instability: Energy is NaN/Inf at step {t_step+1}. Stopping.")
                    numerical_error = True
                    break
                hist_idx += 1
        
        # Save intermediate checkpoint
        if (t_step + 1) % config['checkpoint_every_n_steps'] == 0 and (t_step + 1) < config['T_steps']:
            intermediate_ckpt_file = os.path.join(checkpoint_path_lss, f"intermediate_CKPT_{config['run_id']}_step_{t_step+1}.npz")
            try:
                np.savez_compressed(intermediate_ckpt_file,
                                    phi_r_cpu=phi.cpu().numpy(),
                                    phi_dot_r_cpu=phi_dot.cpu().numpy(),
                                    last_step=t_step,
                                    config_lss_saved=config,
                                    energy_history=energy_history[:hist_idx],
                                    density_norm_history=density_norm_history[:hist_idx])
                print(f"Checkpoint saved at step {t_step+1} to {intermediate_ckpt_file}")
            except Exception as e_save:
                print(f"Error saving intermediate LSS checkpoint: {e_save}")

    sim_duration = time.time() - sim_start_time
    print(f"Simulation finished in {sim_duration:.2f} seconds.")
    if numerical_error: print("Simulation stopped due to numerical error.")

    # Save final state and history
    final_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    final_data_filename = os.path.join(data_path_lss, f"FINAL_LSS_DATA_{config['run_id']}_{final_timestamp}.npz")
    np.savez_compressed(final_data_filename,
                        phi_final_cpu=phi.cpu().numpy(),
                        phi_dot_final_cpu=phi_dot.cpu().numpy(),
                        energy_history=energy_history[:hist_idx],
                        density_norm_history=density_norm_history[:hist_idx],
                        config_lss=config,
                        sim_had_numerical_error=numerical_error)
    print(f"Final LSS simulation data saved to {final_data_filename}")

    del phi, phi_dot, efm_model
    gc.collect()
    torch.cuda.empty_cache()

    return final_data_filename


## Analysis and Plotting

After the simulation, load the saved data to visualize the results, focusing on:
*   **Field Energy Evolution:** To observe stability and trends over the long run.
*   **Density Norm Evolution:** `kφ²` represents mass-energy density in EFM. Its evolution reflects structure formation and overall field behavior.
*   **Power Spectrum P(k):** To identify characteristic clustering scales in k-space.
*   **Correlation Function ξ(r):** To identify characteristic clustering scales in real space.

### Physical Scaling for Interpretation

The simulation runs in dimensionless units. To interpret the results in physical units (Mpc), we need to establish a scaling factor. A common approach is to match a known or predicted physical scale (e.g., the 628 Mpc primary LSS scale from EFM [4]) to an emergent dimensionless scale from the simulation (e.g., the largest peak in P(k) or ξ(r)).

If the simulation yields a primary clustering scale at `r_sim_peak` (dimensionless) that should correspond to `r_phys_peak = 628 Mpc`, then the scaling factor `S_L = r_phys_peak / r_sim_peak`. All other dimensionless lengths can then be converted by multiplying by `S_L`.

Similarly, for k-space, if `k_sim_peak` corresponds to `k_phys_peak = 2π / 628 Mpc⁻¹`, then `S_k = 1/S_L`.

In [ ]:
def plot_lss_results(data_file_path: str):
    print(f"Loading data for plotting from: {data_file_path}")
    try:
        data = np.load(data_file_path, allow_pickle=True)
        energy_history = data['energy_history']
        density_norm_history = data['density_norm_history']
        phi_final_cpu = data['phi_final_cpu']
        config = data['config_lss'].item()
        sim_had_numerical_error = data['sim_had_numerical_error'].item()
        print("Data loaded successfully.")
        if sim_had_numerical_error: print("WARNING: Simulation previously encountered numerical error.")

        num_hist_points = len(energy_history)
        time_sim_unit = np.arange(num_hist_points) * config['history_every_n_steps'] * config['dt_sim_unit']

        plt.figure(figsize=(14, 6))
        plt.subplot(1, 2, 1)
        plt.plot(time_sim_unit, energy_history, marker='.', linestyle='-')
        plt.title('Total Field Energy Evolution (Dimensionless Units)')
        plt.xlabel('Time (Simulation Units)')
        plt.ylabel('Energy (Dimensionless Units)')
        plt.grid(True)
        plt.ticklabel_format(style='sci', axis='y', scilimits=(-3,3), useMathText=True)

        plt.subplot(1, 2, 2)
        plt.plot(time_sim_unit, density_norm_history, marker='.', linestyle='-')
        plt.title('Density Norm (kφ²) Evolution (Dimensionless Units)')
        plt.xlabel('Time (Simulation Units)')
        plt.ylabel('Density Norm (Dimensionless Units)')
        plt.grid(True)
        plt.ticklabel_format(style='sci', axis='y', scilimits=(-3,3), useMathText=True)
        
        plt.suptitle(f"EFM LSS Simulation Results ({config['run_id']})", fontsize=16, y=1.04)
        plt.tight_layout(rect=[0, 0.03, 1, 0.98])
        plot_filename_evo = os.path.join(data_path_lss, f"lss_evo_results_{config['run_id']}.png")
        plt.savefig(plot_filename_evo)
        plt.show()
        plt.close()

        print(f"\n--- Final Simulation Properties ({config['run_id']}) ---")
        print(f"Final Time Simulated: {time_sim_unit[-1]:.4g} Dimensionless Units")
        print(f"Final Field Energy: {energy_history[-1]:.4g}")
        print(f"Final Density Norm (kφ²): {density_norm_history[-1]:.4g}")

        # --- Power Spectrum and Correlation Function Analysis (Dimensionless) ---+
        print("\nComputing P(k) and xi(r) for LSS final state (dimensionless units)...")
        
        k_min_plot_sim = 2 * np.pi / config['L_sim_unit'] * 0.5 # Smallest k to plot (half the fundamental mode)
        k_max_plot_sim = np.pi / config['dx_sim_unit'] * 0.9   # Largest k to plot (90% of Nyquist frequency)
        
        k_bins_sim, pk_vals_sim = compute_power_spectrum_lss(
            phi_final_cpu, k_val_range=[k_min_plot_sim, k_max_plot_sim],
            dx_val_param=config['dx_sim_unit'], N_grid_param=config['N']
        )
        r_bins_sim, xi_vals_sim = compute_correlation_function_lss(
            phi_final_cpu, dx_val_param=config['dx_sim_unit'],
            N_grid_param=config['N'], L_box_param=config['L_sim_unit']
        )
        del phi_final_cpu # Free memory after use
        gc.collect()

        plt.figure(figsize=(16,6))
        
        plt.subplot(1,2,1)
        plt.loglog(k_bins_sim, pk_vals_sim)
        plt.title('LSS Power Spectrum P(k) (Dimensionless Units)')
        plt.xlabel('k (Dimensionless Units)')
        plt.ylabel('P(k) (Dimensionless Units)')
        plt.grid(True, which='both', linestyle=':')
        plt.xlim([k_min_plot_sim, k_max_plot_sim]) # Set explicit x-limits

        plt.subplot(1,2,2)
        plt.plot(r_bins_sim, xi_vals_sim)
        plt.title(r'LSS Correlation Function $\xi$(r) (Dimensionless Units)')
        plt.xlabel('r (Dimensionless Units)')
        plt.ylabel(r'$\xi$(r) (Dimensionless Units)') # Using raw string for LaTeX-like symbol
        plt.grid(True, linestyle=':')
        plt.axhline(0, color='black', linewidth=0.5)
        if len(xi_vals_sim[1:]) > 0 and np.max(np.abs(xi_vals_sim[1:])) > 1e-10:
            abs_max_xi_plot = np.max(np.abs(xi_vals_sim[1:]))
            plt.ylim(max(-1.5 * abs_max_xi_plot, np.min(xi_vals_sim) - 0.1*abs_max_xi_plot ), 
                     min( 1.5 * abs_max_xi_plot, np.max(xi_vals_sim) + 0.1*abs_max_xi_plot ))

        plt.tight_layout()
        plt.suptitle(f"EFM LSS Observables (Dimensionless, {config['run_id']})", fontsize=14, y=1.02)
        plot_filename_obs = os.path.join(data_path_lss, f"lss_observables_{config['run_id']}.png")
        plt.savefig(plot_filename_obs)
        plt.show()
        plt.close()

        # --- Identify and Print Emergent Dimensionless Scales ---
        print("\n--- Emergent Dimensionless Scales from Simulation ---")
        r_sim_peak = "N/A"
        if len(xi_vals_sim) > 1 and np.any(np.abs(xi_vals_sim[1:]) > 1e-10):
            # Find main peak in correlation function (excluding r=0 which is autocorr)
            # Look for the first significant peak or a broad correlation
            # A more robust method would involve smoothing or looking for features
            # For now, just taking the max after initial spike, or a more sophisticated peak detection:
            # To avoid picking noise at small r, let's look for a peak that's significantly above baseline noise
            # Simple method: Find index of max value, excluding r=0. Then check if value is significant.
            if np.max(xi_vals_sim[1:]) > 1e-10: # Check if there's any positive correlation
                # Try to find a prominent peak using a simple approach (e.g., max value)
                # A more sophisticated peak finding (e.g., scipy.signal.find_peaks) might be needed for subtle features.
                max_val_idx = np.argmax(xi_vals_sim[1:]) + 1 # Index of max value after r=0
                if xi_vals_sim[max_val_idx] > 3 * np.mean(xi_vals_sim[-10:]): # Simple check for prominence
                    r_sim_peak = r_bins_sim[max_val_idx]
                    print(f"Main correlation peak in dimensionless units (r_sim_peak): {r_sim_peak:.3f}")
                else:
                    print("No prominent correlation peak found in xi(r) beyond noise level.")
            else:
                print("Correlation function is effectively zero or flat.")
        else:
            print("Correlation function is effectively zero or flat.")

        k_sim_peak = "N/A"
        if len(pk_vals_sim) > 0 and np.any(pk_vals_sim > 1e-10):
            # Find main peak in power spectrum
            k_sim_peak = k_bins_sim[np.argmax(pk_vals_sim)]
            print(f"Main power spectrum peak in dimensionless units (k_sim_peak): {k_sim_peak:.3f}")
            print(f"Corresponding length scale (2π/k_sim_peak): {2*np.pi/k_sim_peak:.3f}")
        else:
            print("Power spectrum is effectively zero.")

        print("\n--- Physical Interpretation (based on EFM theoretical predictions) ---")
        # Assume EFM's primary LSS scale (628 Mpc) corresponds to the largest emergent scale from simulation
        # This is where the physical world is 'mapped' to the dimensionless simulation.
        EFM_PRIMARY_LSS_Mpc = 628.0 # Mpc [4]
        EFM_SECONDARY_LSS_Mpc = 147.0 # Mpc (BAO-like) [4]

        if r_sim_peak != "N/A" and isinstance(r_sim_peak, float): # Check if a peak was found and is a float
            scaling_factor_L = EFM_PRIMARY_LSS_Mpc / r_sim_peak
            print(f"Scaling factor (1 dimensionless unit = X Mpc) derived from 628 Mpc primary scale: {scaling_factor_L:.2e} Mpc/unit")
            
            # Physical constants for time scaling
            c_si_m_s = 299792458.0 # m/s
            Mpc_to_m = 3.08567758e22 # meters per Mpc
            s_to_yr = 1.0 / (3.15576e7) # years per second

            # Re-plot with physical units on x-axis
            plt.figure(figsize=(16,6))
            
            # P(k) in physical units
            plt.subplot(1,2,1)
            # k_phys = k_sim / L_scale_factor; P_phys = P_sim * L_scale_factor^3
            plt.loglog(k_bins_sim / scaling_factor_L, pk_vals_sim * scaling_factor_L**3) # P(k) ~ L^3
            plt.title(r'LSS Power Spectrum P(k) (Physical Units)')
            plt.xlabel(r'k (Mpc$^{-1}$)')
            plt.ylabel(r'P(k) (Mpc$^3$)')
            plt.grid(True, which='both', linestyle=':')
            plt.axvline(2*np.pi/EFM_PRIMARY_LSS_Mpc, color='g', linestyle='--', label=f'EFM {EFM_PRIMARY_LSS_Mpc} Mpc (Primary)')
            plt.axvline(2*np.pi/EFM_SECONDARY_LSS_Mpc, color='r', linestyle='--', label=f'EFM {EFM_SECONDARY_LSS_Mpc} Mpc (BAO-like)')
            plt.legend()

            # xi(r) in physical units
            plt.subplot(1,2,2)
            plt.plot(r_bins_sim * scaling_factor_L, xi_vals_sim)
            plt.title(r'LSS Correlation Function $\xi$(r) (Physical Units)')
            plt.xlabel('r (Mpc)')
            plt.ylabel(r'$\xi$(r) (Dimensionless Units)') # ξ is dimensionless
            plt.grid(True, linestyle=':')
            plt.axhline(0, color='black', linewidth=0.5)
            plt.axvline(EFM_PRIMARY_LSS_Mpc, color='g', linestyle='--', label=f'EFM {EFM_PRIMARY_LSS_Mpc} Mpc (Primary)')
            plt.axvline(EFM_SECONDARY_LSS_Mpc, color='r', linestyle='--', label=f'EFM {EFM_SECONDARY_LSS_Mpc} Mpc (BAO-like)')
            if len(xi_vals_sim[1:]) > 0 and np.max(np.abs(xi_vals_sim[1:])) > 1e-10: # Reuse y-limits if meaningful
                 abs_max_xi_plot = np.max(np.abs(xi_vals_sim[1:]))
                 plt.ylim(max(-1.5 * abs_max_xi_plot, np.min(xi_vals_sim) - 0.1*abs_max_xi_plot ), 
                         min( 1.5 * abs_max_xi_plot, np.max(xi_vals_sim) + 0.1*abs_max_xi_plot ))
            plt.legend()

            plt.tight_layout()
            plt.suptitle(f"EFM LSS Observables (Physical, {config['run_id']})", fontsize=14, y=1.02)
            plot_filename_obs_phys = os.path.join(data_path_lss, f"lss_observables_physical_{config['run_id']}.png")
            plt.savefig(plot_filename_obs_phys)
            plt.show()
            plt.close()
            
            # Approx physical time duration calculation
            # 1 dimensionless length unit corresponds to scaling_factor_L Mpc
            # 1 dimensionless time unit = (1 dimensionless length unit / c_sim_unit) = (scaling_factor_L * Mpc_to_m / c_si_m_s) in seconds
            # Convert to Gyr:
            approx_physical_time_per_sim_unit = (scaling_factor_L * Mpc_to_m / c_si_m_s) / s_to_yr # in years
            approx_total_physical_time_gyr = time_sim_unit[-1] * approx_physical_time_per_sim_unit / 1e9

            print(f"Physical box size L: {config['L_sim_unit'] * scaling_factor_L:.2f} Mpc")
            print(f"Physical dx: {config['dx_sim_unit'] * scaling_factor_L:.4g} Mpc")
            print(f"Approx. total physical time simulated: {approx_total_physical_time_gyr:.4g} Gyr")

        else:
            print("Could not determine a primary dimensionless peak for physical scaling (xi(r) peak was not found). Please examine raw P(k) and xi(r) plots for subtle features.")

    except Exception as e:
        print(f"Error during plotting/analysis: {e}")


if __name__ == '__main__':
    # Run the simulation
    # This single-process run leverages the full A100 GPU for simplicity.
    if torch.cuda.is_available():
        main_device = torch.device('cuda:0')
    else:
        main_device = torch.device('cpu')
    
    final_data_file = run_lss_simulation(config_lss_run, main_device)
    
    # Plot the results
    if final_data_file:
        plot_lss_results(final_data_file)


## Conclusion and Future Work

This notebook presents an updated simulation for EFM's Large-Scale Structure formation using a consistent dimensionless parameter set on an A100 GPU. The aim is to achieve numerical stability and observe the emergence of characteristic clustering scales (like 147 Mpc and 628 Mpc) as predicted by EFM theory.

**Key Objectives and Expected Outcomes:**

*   **Numerical Stability:** The dimensionless approach, combined with appropriate CFL factors, should ensure the simulation runs for the full duration without `NaN/Inf` errors. (Achieved!)
*   **Emergent Clustering Scales:** We expect to see peaks in the Power Spectrum and Correlation Function corresponding to the 147 Mpc (BAO-like) and 628 Mpc (primary LSS) scales after physical scaling. This would provide strong computational validation for EFM's unique LSS predictions.
*   **Dynamic Field Evolution:** The energy and density norm plots should demonstrate a stable or dynamically evolving field, consistent with the self-organization mechanisms of EFM. (Achieved, showing stable oscillations after initial transient!)

**Analysis of Current Results and Remaining Challenges:**

The current run (N=300, T=50000, dimensionless parameters) successfully completed and is numerically stable. The energy and density norm plots show initial transients settling into a stable, oscillatory state. However, the P(k) and ξ(r) plots do not yet show clear, distinct peaks corresponding to the 147 Mpc and 628 Mpc scales. This suggests that while the field is stable and evolving, the specific conditions for robust LSS *structure formation* (i.e., patterns that are distinguishable from initial noise or weak fluctuations) may not be fully met or require further refinement.

Possible reasons for the lack of clear peaks:
*   **Initial Noise vs. Seeded Perturbations:** Simple Gaussian noise might not be sufficient to reliably seed the large-scale structures expected. Some LSS simulations use specific low-amplitude, long-wavelength sinusoidal modes as initial conditions to promote structure growth.
*   **Simulation Time:** Even 50,000 steps might not be long enough for *detectable* large-scale structures to fully coalesce and amplify on a 300³ grid given the chosen interaction strengths.
*   **Parameter Fine-tuning:** While the parameters are from the paper, the precise interplay of `g`, `k_gravity`, `alpha`, and `delta` might need more fine-tuning for optimal structure emergence. For example, the `g` value might need to be higher to drive stronger non-linear clumping, or `alpha`/`delta` might be damping too much.
*   **Small Initial Amplitude:** `1.0e-6` might be too small to sufficiently perturb the field to initiate strong structure formation without excessive runtime.

**Future Work and Next Steps (Leveraging HPC & Grounding):**

1.  **Iterative Parameter Adjustment & Longer Runs (HPC):** Given numerical stability, the next step is to continue iterating on the dimensionless parameters (`g_sim`, `k_efm_gravity_coupling`, `alpha_sim`, `delta_sim`, `initial_noise_amplitude`) and significantly increasing `T_steps` (e.g., to 100,000 or more) on the H100 cluster. We need to find the sweet spot where these terms lead to visible large-scale patterns in the field, not just stable oscillations. This would involve a parameter sweep if possible.
2.  **Seeding Initial Perturbations:** Experiment with different forms of initial conditions. Instead of pure noise, consider adding small, long-wavelength sinusoidal perturbations (e.g., `φ_initial = A * (sin(k_x x) + sin(k_y y) + sin(k_z z))`) at the target dimensionless scales that would map to 147 Mpc and 628 Mpc.
3.  **Visualizing Field Slices:** Implement a function to save 2D slices or 3D renderings of the `phi_final_cpu` field to visually inspect for any emergent filamentary or clustering patterns, even if weak.
4.  **Unit System Derivation:** The need for a concrete derivation of the fundamental unit scaling within EFM from first principles (e.g., `s*t=k`, HDS density levels) remains crucial to rigorously map dimensionless results to physical Mpc/Gyr. This is a separate theoretical task that complements the simulations.
5.  **Integration with other EFM Cosmological Aspects:** Combine the LSS simulation with the inflationary analogue and Hubble tension resolution mechanisms into a more comprehensive end-to-end EFM cosmological simulation.

This systematic approach, grounded in EFM's dimensionless framework, will build a robust, falsifiable, and unified cosmological model, providing a deterministic alternative to current paradigms. 


## References

[1] Emvula, T. 2025a, 'Introducing the Ehokolo Fluxon Model: A Scalar Motion Framework for the Physical Universe' (Independent Frontier Science Collaboration, April 2025).
[2] Emvula, T. 2025b, 'The Ehokolo Fluxon Model: A Foundation for Physics from Eholokon Dynamics' (Independent Frontier Science Collaboration, April 2025).
[3] Emvula, T. 2025c, 'Ehokolo Fluxon Model: Mass Generation via Ehokolon Self-Interactions' (Independent Frontier Science Collaboration, March 2025).
[4] Emvula, T. 2025d, 'Ehokolo Fluxon Model: Unifying Cosmic Structure, Non-Gaussianity, and Gravitational Waves Across Scales' (Independent Frontier Science Collaboration, April 2025). (This paper is the primary reference for LSS parameters)
[5] Emvula, T. 2025e, 'Fluxonic Zero-Point Energy and Emergent Gravity: A Deterministic Alternative to Spacetime Curvature in the Ehokolo Fluxon Model' (Independent Frontier Science Collaboration, February 2025).
[6] Emvula, T. 2025f, 'Fluxonic Cosmology: Inflation, Expansion, and Structure from EFM Harmonic States' (Independent Frontier Science Collaboration, April 2025).
[7] Emvula, T. 2025g, 'Ehokolon Harmonic Density States: Foundational Validation and Unified Physics in the Ehokolo Fluxon Model' (Independent Frontier Science Collaboration, April 2025).
